**{DEV}\[TODO\]**
- everything
- get rid of GetXY
- work out rotate function

---
### Chapter 1: System Modelling
---
# **Tutorial 1.d: Pendulum in 3D**
**Aim:**

### **Contents**:
* [3D Modelling](#3D-Modelling)
* [Spherical Pendulum](#Spherical-Pendulum)
* [Hinge Pendulum](#Hinge-Pendulum)

## **3D Modelling**

In [1]:
# import libraries
import sympy as sym
import numpy as np
import time # for benchmarking different models

from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>"))

# define time-logging function for benchmarking
def log_time(t_start=None, msg=""):
    t_end = time.perf_counter()
    if t_start == None:
        if msg == "": print("Timer initialized")
        else: print(msg)
    else: print(msg + f": {t_end - t_start:.6f} sec")
    return t_end

# create symbolic variables - Generalized for n-link
nlinks = 2
links = np.arange(nlinks)

# --------------------------------------------------------------------------------------------------------------

# system parameters
g = sym.symbols('g')

m  = [sym.symbols( 'm_{%s}' %(i+1)) for i in links] # mass of links
l  = [sym.symbols( 'l_{%s}' %(i+1)) for i in links] # length of links
d  = [sym.symbols( 'd_{%s}' %(i+1)) for i in links] # distance to COM of links (from origin)

I_psi = [sym.symbols('I_{\\psi_{%s}}'   %(i+1)) for i in links] # moment of intertia of links
I_th  = [sym.symbols('I_{\\theta_{%s}}' %(i+1)) for i in links]
I_phi = [sym.symbols('I_{\\phi_{%s}}'   %(i+1)) for i in links]

# generalized coordinates
X0, Y0, Z0 = sym.symbols(['X_{0}','Y_{0}', 'Z_{0}']) # fixed position of first link

psi   = [sym.symbols(       '\\psi_{%s}' %(i+1)) for i in links] #positions
dpsi  = [sym.symbols( '\dot{\\psi}_{%s}' %(i+1)) for i in links] #velocities
ddpsi = [sym.symbols('\ddot{\\psi}_{%s}' %(i+1)) for i in links] #accelerations

th   = [sym.symbols(       '\\theta_{%s}' %(i+1)) for i in links] #positions
dth  = [sym.symbols( '\dot{\\theta}_{%s}' %(i+1)) for i in links] #velocities
ddth = [sym.symbols('\ddot{\\theta}_{%s}' %(i+1)) for i in links] #accelerations

phi   = [sym.symbols(       '\\phi_{%s}' %(i+1)) for i in links] #positions
dphi  = [sym.symbols( '\dot{\\phi}_{%s}' %(i+1)) for i in links] #velocities
ddphi = [sym.symbols('\ddot{\\phi}_{%s}' %(i+1)) for i in links] #accelerations

q   = sym.Matrix([[  psi[i],  th[i],  phi[i]] for i in links]) # fixed base so only rotations
dq  = sym.Matrix([[ dpsi[i], dth[i], dphi[i]] for i in links])
ddq = sym.Matrix([[ddpsi[i],ddth[i],ddphi[i]] for i in links])

q   =   q.reshape(q.rows * q.cols, 1)
dq  =  dq.reshape(q.rows * q.cols, 1)
ddq = ddq.reshape(q.rows * q.cols, 1)

display(q)
# --------------------------------------------------------------------------------------------------------------

# STEP 1: system space coordinates written in terms of the generalised coordinates

# helper functions
# the 3D system space coordinates are [x;y;z;psi;th;phi].
# It is easier to split up the translations and orientations into separate variables in 3D
order = ['z','y','x'] # rotation order

Matrix([
[  \psi_{1}],
[\theta_{1}],
[  \phi_{1}],
[  \psi_{2}],
[\theta_{2}],
[  \phi_{2}]])

In [26]:
# COPIED FROM RTAO pendulum, IN PROGRESS

def get_RE(phi, th, psi, order): # generate rotation matrix
    o1, o2, o3 = order
    
    # Define rotation matrices for each axis of rotation
    Rx = sym.Matrix([[1,            0,             0],
                     [0, sym.cos(phi), -sym.sin(phi)],
                     [0, sym.sin(phi),  sym.cos(phi)]])

    Ry = sym.Matrix([[ sym.cos(th), 0, sym.sin(th)],
                     [           0, 1,           0],
                     [-sym.sin(th), 0, sym.cos(th)]])

    Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                     [sym.sin(psi),  sym.cos(psi), 0],
                     [           0,             0, 1]])
    
    # map the rotation matrices to their axes
    Rdict = {'x': Rx,
             'y': Ry,
             'z': Rz}
    
    # Define rotation matrices for moving between the body frame and each of the previous frames
    R32 = Rdict[o3]     # undo last 1 rotation
    R31 = Rdict[o2]*R32 # undo last 2 rotations
    R30 = Rdict[o1]*R31 # undo  all 3 rotations
    
    # Euler Angle Rates
    
    # Define axis vectors in their own frames
    axes = {'x': sym.Matrix([[1],[0],[0]]),
            'y': sym.Matrix([[0],[1],[0]]),
            'z': sym.Matrix([[0],[0],[1]])}
    
    # Transform these axes so that they are all in the BODY frame *not inertial
    E_cols = {o3: R32.transpose()*axes[o3],
              o2: R31.transpose()*axes[o2],
              o1: R30.transpose()*axes[o1]}
    
    # Rearrange axes into the order x,y,z (so that this matrix can multiply by vectors of the form x,y,z)
    E = sym.Matrix.hstack(E_cols['x'], E_cols['y'], E_cols['z']) #hstack -> horizontally stack
    # E = sym.Matrix([E_cols['x'], E_cols['y'], E_cols['z']]).reshape(3,3).transpose() # does the same thing as above
    
    return [R30, E] # return 1) the rotation from body to inertial, and 2) the euler rates matrix

Matrix([
[             -sin(\theta_{1})],
[sin(\phi_{1})*cos(\theta_{1})],
[cos(\phi_{1})*cos(\theta_{1})]])

Matrix([
[1,              0,              -sin(\theta_{1})],
[0,  cos(\phi_{1}), sin(\phi_{1})*cos(\theta_{1})],
[0, -sin(\phi_{1}), cos(\phi_{1})*cos(\theta_{1})]])

In [ ]:
# positions of each link in their own reference frames
rCOMs_0 = []
rEnds_0 = [sym.Matrix([[X0],[Y0],[Z0]])] # positions of the tops/bottoms of links

R, E = [0]*nlinks, [0]*nlinks

for i in range(nlinks): # enumerate returns the index and the value at that index
    R[i], E[i] = get_RE(phi[i], th[i], psi[i], order)
    
    rLen_n = sym.Matrix([[0],[0],[-l[i]]])
    rTop_0 = rEnds_0[i]

    rCOMs_0.append( R[i]*rLen_n*d[i] + rTop_0)
    rEnds_0.append( R[i]*rLen_n      + rTop_0) # indexing: top of link matches COM in rCOM_0. extra row in rEnds_0 = bottom of nth link



In [38]:
c = [1]*5
for i in range(5):
    c[i] = i+1
    print(c[i], end=' ')
    c[i] = np.prod([c[j] for j in range(i+1)])
    print(c[i])

1 1
2 2
3 6
4 48
5 2880


## **Spherical Pendulum**

## **Hinge Pendulum**